In [1]:
import pandas as pd
import numpy as np

In [2]:
def maze_summary(dataframe, category):
    '''
    Calculate the means and standard deviations of certain categories in the dataframe.
    dataframe: data table, pandas dataframe
    category: user specified categories for calculation, python list
    '''
    maze_list = dataframe['maze'].unique().tolist()
    mode_list = dataframe['mode'].unique().tolist()
    intuition_list = dataframe['intuition'].unique().tolist()
    heuristic_list = dataframe['heuristic'].unique().tolist()
    summary_mean = pd.DataFrame()
    summary_std = pd.DataFrame()
    for maze in maze_list:
        for mode in mode_list:
            for intuition in intuition_list:
                for heuristic in heuristic_list:
                    data = dataframe[(dataframe['maze'] == maze) & (dataframe['mode'] == mode) & (dataframe['intuition'] == intuition) & (dataframe['heuristic'] == heuristic)][category]
                    data_mean = np.around(data.mean().values, decimals = 2).tolist()
                    data_std = np.around(data.std().values, decimals = 2).tolist()
                    summary_mean = summary_mean.append([[maze, mode, intuition, heuristic] + data_mean], ignore_index = True)
                    summary_std = summary_std.append([[maze, mode, intuition, heuristic] + data_std], ignore_index = True)
    return (summary_mean, summary_std)

In [3]:
def file_process(file_list):
    '''
    Process files for maze_summary.
    file_list: file name list, python list
    '''
    summary_mean = pd.DataFrame()
    summary_std = pd.DataFrame()
    for file in file_list:
        dataframe = pd.read_csv(file, sep = ',')
        category = dataframe.columns.values.tolist()[4:]
        summary = maze_summary(dataframe = dataframe, category = category)
        summary_mean = summary_mean.append(summary[0], ignore_index = True)
        summary_std = summary_std.append(summary[1], ignore_index = True)
    return (summary_mean, summary_std)

In [4]:
def summary_process(dataframe_list):
    '''
    Merge statistics of multiple dataframe in a list.
    dataframe_list: dataframe list, python list
    '''
    summary_mean = pd.DataFrame()
    summary_std = pd.DataFrame()
    for dataframe in dataframe_list:
        category = dataframe.columns.values.tolist()[4:]
        summary = maze_summary(dataframe = dataframe, category = category)
        summary_mean = summary_mean.append(summary[0], ignore_index = True)
        summary_std = summary_std.append(summary[1], ignore_index = True)
    return (summary_mean, summary_std)

In [5]:
file_list = ['test_result_maze_01.csv', 'test_result_maze_02.csv', 'test_result_maze_03.csv']

In [6]:
dataframe_list = list()
for file in file_list:
    dataframe_list.append(pd.read_csv(file, sep = ','))

In [7]:
# Take a look at the data
dataframe_list[0].head()

,maze,mode,intuition,heuristic,num_actions_1,length_movement_1,num_actions_2,length_movement_2,num_actions_3,length_movement_3,true_coverage,score,exploration_time,computation_time
0,1,complete,False,False,183.0,222.0,2.0,4.0,17.0,32.0,1.0,23.733333,436.140,436.535
1,1,complete,False,False,185.0,220.0,14.0,24.0,17.0,32.0,1.0,24.200000,333.405,333.821
2,1,complete,False,False,185.0,218.0,11.0,24.0,17.0,32.0,1.0,24.100000,481.363,481.771
3,1,complete,False,False,188.0,221.0,7.0,13.0,17.0,32.0,1.0,24.066667,420.779,421.243
4,1,complete,False,False,180.0,205.0,9.0,17.0,17.0,32.0,1.0,23.866667,302.459,302.943


In [8]:
# Add maze_size information
# Calculate exploration efficiency
maze_size_dict = {1: 144, 2: 196, 3: 256}
for i in xrange(len(dataframe_list)):
    dataframe_list[i]['maze_size'] = dataframe_list[i]['maze'].map(lambda x: maze_size_dict[x])
    dataframe_list[i]['num_grid_per_action_1'] = dataframe_list[i]['maze_size'] * dataframe_list[i]['true_coverage'] / dataframe_list[i]['num_actions_1']
    dataframe_list[i]['num_grid_per_length_1'] = dataframe_list[i]['maze_size'] * dataframe_list[i]['true_coverage'] / dataframe_list[i]['length_movement_1']

In [9]:
dataframe_list[0].head()

,maze,mode,intuition,heuristic,num_actions_1,length_movement_1,num_actions_2,length_movement_2,num_actions_3,length_movement_3,true_coverage,score,exploration_time,computation_time,maze_size,num_grid_per_action_1,num_grid_per_length_1
0,1,complete,False,False,183.0,222.0,2.0,4.0,17.0,32.0,1.0,23.733333,436.140,436.535,144,0.786885,0.648649
1,1,complete,False,False,185.0,220.0,14.0,24.0,17.0,32.0,1.0,24.200000,333.405,333.821,144,0.778378,0.654545
2,1,complete,False,False,185.0,218.0,11.0,24.0,17.0,32.0,1.0,24.100000,481.363,481.771,144,0.778378,0.660550
3,1,complete,False,False,188.0,221.0,7.0,13.0,17.0,32.0,1.0,24.066667,420.779,421.243,144,0.765957,0.651584
4,1,complete,False,False,180.0,205.0,9.0,17.0,17.0,32.0,1.0,23.866667,302.459,302.943,144,0.800000,0.702439


In [10]:
column_names = ['Maze', 'Mode', 'Intuition', 'Heuristics', 'Time Steps 1', 'Path Length 1', 
                'Time Steps 2', 'Path Length 2', 'Time Steps 3', 'Path Length 3','Coverage', 'Score', 'Exploration Time', 'Computation Time', 'Maze Size', 'Number of Grid Visited Per Time Step during Exploration', 'Number of Grid Visited Per Path Length during Exploration']

In [11]:
summary = summary_process(dataframe_list)
summary_mean = summary[0]
summary_std = summary[1]

In [12]:
summary_mean.columns = column_names
summary_std.columns = column_names

In [13]:
summary_mean

,Maze,Mode,Intuition,Heuristics,Time Steps 1,Path Length 1,Time Steps 2,Path Length 2,Time Steps 3,Path Length 3,Coverage,Score,Exploration Time,Computation Time,Maze Size,Number of Grid Visited Per Time Step during Exploration,Number of Grid Visited Per Path Length during Exploration
0,1,complete,False,False,186.9,222.2,6.9,13.6,17.0,32.0,1.00,24.03,411.58,412.02,144.0,0.77,0.65
1,1,complete,False,True,185.6,217.3,8.3,15.3,17.0,32.0,1.00,24.03,96.16,96.54,144.0,0.78,0.66
2,1,complete,True,False,188.3,222.1,9.8,17.7,17.0,32.0,1.00,24.17,102.46,102.86,144.0,0.77,0.65
3,1,complete,True,True,189.3,220.9,9.0,17.7,17.0,32.0,1.00,24.18,45.33,45.73,144.0,0.76,0.65
4,1,incomplete,False,False,107.5,122.6,18.2,31.6,18.2,32.4,0.65,23.00,211.81,212.18,144.0,0.89,0.80
5,1,incomplete,False,True,134.1,153.4,17.5,31.0,17.5,32.6,0.78,23.14,67.76,68.13,144.0,0.85,0.75
6,1,incomplete,True,False,105.0,119.1,18.1,31.7,18.1,32.9,0.64,22.81,46.06,46.42,144.0,0.89,0.79
7,1,incomplete,True,True,120.6,135.0,18.3,31.0,18.3,32.2,0.71,23.54,28.56,28.93,144.0,0.86,0.78
8,2,complete,False,False,261.1,310.6,12.0,21.0,22.0,43.0,1.00,31.84,752.10,752.69,196.0,0.75,0.63
9,2,complete,False,True,266.4,322.3,13.1,23.9,22.0,43.0,1.00,32.05,181.74,182.29,196.0,0.74,0.61


In [14]:
summary_std

,Maze,Mode,Intuition,Heuristics,Time Steps 1,Path Length 1,Time Steps 2,Path Length 2,Time Steps 3,Path Length 3,Coverage,Score,Exploration Time,Computation Time,Maze Size,Number of Grid Visited Per Time Step during Exploration,Number of Grid Visited Per Path Length during Exploration
0,1,complete,False,False,3.81,8.23,3.70,6.57,0.00,0.00,0.00,0.15,99.95,99.96,0.0,0.02,0.02
1,1,complete,False,True,7.44,12.21,3.33,6.62,0.00,0.00,0.00,0.21,6.91,6.90,0.0,0.03,0.04
2,1,complete,True,False,5.95,9.46,4.73,7.94,0.00,0.00,0.00,0.33,17.19,17.19,0.0,0.02,0.03
3,1,complete,True,True,9.68,14.72,5.98,10.31,0.00,0.00,0.00,0.34,2.97,2.97,0.0,0.04,0.04
4,1,incomplete,False,False,43.39,56.70,0.92,1.26,0.92,1.58,0.21,1.45,96.64,96.65,0.0,0.07,0.10
5,1,incomplete,False,True,38.87,50.78,0.85,1.70,0.85,1.35,0.17,1.26,23.80,23.80,0.0,0.06,0.09
6,1,incomplete,True,False,24.25,29.91,1.20,2.11,1.20,2.13,0.12,1.36,23.41,23.42,0.0,0.06,0.08
7,1,incomplete,True,True,41.46,50.14,1.49,1.05,1.49,0.63,0.21,1.31,13.01,13.02,0.0,0.06,0.09
8,2,complete,False,False,8.31,12.69,6.83,13.08,0.00,0.00,0.00,0.39,201.85,201.82,0.0,0.02,0.02
9,2,complete,False,True,9.71,18.76,5.51,9.86,0.00,0.00,0.00,0.44,11.02,11.02,0.0,0.03,0.04


In [15]:
summary_mean.to_csv('summary_mean.csv', sep = ',', index = False)
summary_std.to_csv('summary_std.csv', sep = ',', index = False)